In [2]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('rdd-test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 14:33:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 14:33:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/03 14:33:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
df_green = spark.read.parquet('data/pq/green/*/*')

### replicate below query in RDD

```
--for reference
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [8]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [12]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), PULocationID=236, total_amount=12.25),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), PULocationID=166, total_amount=5.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 14, 25, 16), PULocationID=152, total_amount=17.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 9, 46), PULocationID=51, total_amount=22.01)]

### handle this condition   lpep_pickup_datetime >= '2020-01-01 00:00:00'

In [20]:
from datetime import datetime

In [22]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [24]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8)

#### prepare for group by hour and zone

In [27]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

### calculate the total revenue by summing the value together

In [31]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

### run rdd and check

In [34]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(5)

[((datetime.datetime(2020, 1, 21, 13, 0), 33), (535.8900000000001, 26)),
 ((datetime.datetime(2020, 1, 18, 11, 0), 128), (33.51, 1)),
 ((datetime.datetime(2020, 1, 19, 9, 0), 166), (782.1999999999998, 46)),
 ((datetime.datetime(2020, 1, 15, 21, 0), 42), (336.1400000000001, 23)),
 ((datetime.datetime(2020, 1, 4, 12, 0), 116), (119.85, 8))]

### map to dataframe row with schema

In [37]:
from collections import namedtuple

In [39]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [41]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [43]:
from pyspark.sql import types

In [49]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [51]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [53]:
df_result.write.parquet('tmp/green-revenue')


24/03/02 23:31:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### RDD model

In [13]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

In [15]:
duration_rdd = df_green \
    .select(columns) \
    .rdd

In [25]:
duration_rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, DOLocationID=41, trip_distance=5.22),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), PULocationID=236, DOLocationID=263, trip_distance=0.87),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), PULocationID=166, DOLocationID=166, trip_distance=0.63),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 14, 25, 16), PULocationID=152, DOLocationID=238, trip_distance=2.71),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 9, 46), PULocationID=51, DOLocationID=3, trip_distance=2.13)]

In [17]:
import pandas as pd


In [27]:
rows = duration_rdd.take(10)

In [29]:
df = pd.DataFrame(rows, columns=columns)

### dummy model

In [32]:
def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [34]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [36]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [38]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|26.099999999999998|
|              4.35|
|              3.15|
|             13.55|
|10.649999999999999|
|             89.65|
|120.19999999999999|
|              7.25|
|54.699999999999996|
|              15.9|
|               9.3|
|              8.75|
|             94.75|
|              46.2|
|             16.85|
|              14.6|
|             25.65|
|               5.0|
|2.9499999999999997|
|               0.0|
+------------------+
only showing top 20 rows

